<a href="https://colab.research.google.com/github/Polinka1304/HW/blob/main/sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import sqlite3

In [2]:
#Конвертация с формата csv в датафрейм
df = pd.read_csv('test_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   uranium_lead_ratio          1100 non-null   float64
 1   carbon_14_ratio             1100 non-null   float64
 2   radioactive_decay_series    1100 non-null   float64
 3   stratigraphic_layer_depth   1100 non-null   float64
 4   geological_period           1100 non-null   object 
 5   paleomagnetic_data          1100 non-null   object 
 6   inclusion_of_other_fossils  1100 non-null   bool   
 7   isotopic_composition        1100 non-null   float64
 8   surrounding_rock_type       1100 non-null   object 
 9   stratigraphic_position      1100 non-null   object 
 10  fossil_size                 1100 non-null   float64
 11  fossil_weight               1100 non-null   float64
 12  age                         1100 non-null   int64  
dtypes: bool(1), float64(7), int64(1),

In [3]:
#Создание базы данных fossil_age.db
conn = sqlite3.connect('fossil_age.db')
cursor = conn.cursor()

In [4]:
#Создание таблицы fossil в БД fossil_age.db
cursor.execute('''CREATE TABLE IF NOT EXISTS fossil (
               id INTEGER PRIMARY KEY,
               uranium_lead_ratio FLOAT,
               carbon_14_ratio FLOAT,
               radioactive_decay_series FLOAT,
               stratigraphic_layer_depth FLOAT,
               geological_period TEXT,
               paleomagnetic_data TEXT,
               inclusion_of_other_fossils BINARY,
               isotopic_composition FLOAT,
               surrounding_rock_type TEXT,
               stratigraphic_position TEXT,
               fossil_size FLOAT,
               fossil_weight FLOAT,
               age INTEGER
               )''')

In [81]:
#Подготовка данных перед импортом1 - Стандартизация
numerical_features = ['uranium_lead_ratio', 'carbon_14_ratio', 'radioactive_decay_series',
       'stratigraphic_layer_depth', 'fossil_size','fossil_weight',
       'fossil_weight']
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [17]:
#Подготовка данных перед импортом
df["inclusion_of_other_fossils"] = df["inclusion_of_other_fossils"].astype(int)


In [18]:
#Импорт данных в таблицу
for index, row in df.iterrows():
    cursor.execute('''INSERT INTO fossil (
                   uranium_lead_ratio,
                   carbon_14_ratio,
                   radioactive_decay_series,
                   stratigraphic_layer_depth,
                   geological_period,
                   paleomagnetic_data,
                   inclusion_of_other_fossils,
                   isotopic_composition,
                   surrounding_rock_type,
                   stratigraphic_position,
                   fossil_size,
                   fossil_weight,
                   age) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', row)


In [49]:
#Корреляция возраста и содержания 14C
#14C в останках постепенно распадается, и по его остаточной удельной активности можно оценить время гибели организма
cursor.execute('''SELECT age,ROUND(carbon_14_ratio,2) FROM fossil ORDER BY carbon_14_ratio LIMIT 10''')
cursor.fetchall()

[(26473, -1.8),
 (24069, -1.8),
 (25263, -1.8),
 (48800, -1.8),
 (39792, -1.79),
 (28792, -1.79),
 (28913, -1.78),
 (33840, -1.78),
 (22851, -1.78),
 (33777, -1.77)]

In [51]:
#Уран-свинцовый метод: основан на определении того, какая доля урана успела распасться за время существования объекта
#Чем больше урана распалось, тем больше соотношение уран/свинец - тем старше останки
cursor.execute('''SELECT age,ROUND(uranium_lead_ratio,2) FROM fossil ORDER BY uranium_lead_ratio DESC LIMIT 10''')
cursor.fetchall()

[(78347, 3.21),
 (65287, 2.76),
 (82135, 2.71),
 (67498, 2.7),
 (75203, 2.66),
 (93741, 2.64),
 (75703, 2.64),
 (75600, 2.62),
 (61872, 2.61),
 (90799, 2.59)]

In [50]:
#Уран-свинцовый метод2
cursor.execute('''SELECT age,ROUND(uranium_lead_ratio,2) FROM fossil ORDER BY uranium_lead_ratio LIMIT 10''')
cursor.fetchall()

[(20897, -1.92),
 (7916, -1.91),
 (18491, -1.91),
 (19294, -1.9),
 (15381, -1.88),
 (10690, -1.88),
 (26175, -1.88),
 (8427, -1.87),
 (13309, -1.87),
 (11461, -1.86)]

In [41]:
#Вывод количества данных по геологическому периоду
cursor.execute('''SELECT COUNT(*), geological_period FROM fossil GROUP BY geological_period''')
cursor.fetchall()

[(426, 'Cambrian'),
 (20, 'Carboniferous'),
 (332, 'Cretaceous'),
 (220, 'Devonian'),
 (240, 'Jurassic'),
 (142, 'Neogene'),
 (50, 'Ordovician'),
 (216, 'Paleogene'),
 (210, 'Permian'),
 (10, 'Silurian'),
 (334, 'Triassic')]

In [52]:
#Вывод средних значений для останков по уран-свинцовому содержанию и углерода 14 сгруппированных по геологическому периоду
cursor.execute('''SELECT ROUND(AVG(uranium_lead_ratio),2), ROUND(AVG(carbon_14_ratio),2), geological_period FROM fossil GROUP BY geological_period''')
cursor.fetchall()

[(0.28, 0.19, 'Cambrian'),
 (0.43, 0.39, 'Carboniferous'),
 (0.23, 0.16, 'Cretaceous'),
 (0.27, 0.28, 'Devonian'),
 (0.25, 0.15, 'Jurassic'),
 (0.19, 0.3, 'Neogene'),
 (0.12, 0.28, 'Ordovician'),
 (0.27, 0.27, 'Paleogene'),
 (0.32, 0.25, 'Permian'),
 (0.74, 0.09, 'Silurian'),
 (0.17, 0.24, 'Triassic')]

In [77]:
for ax, feature in zip(axes.reshape(-1), numerical_features):
  print(ax)

Axes(0.125,0.53;0.227941x0.35)
Axes(0.398529,0.53;0.227941x0.35)
Axes(0.672059,0.53;0.227941x0.35)
Axes(0.125,0.11;0.227941x0.35)
Axes(0.398529,0.11;0.227941x0.35)
Axes(0.672059,0.11;0.227941x0.35)


In [86]:
#Визуализация численных признаков

for feature in numerical_features:
  df['age'].hist(df[feature], alpha=0.5, figsize=(15,15),xrot=45 ,bins=10)
  plt.xlabel('Age')
  plt.ylabel(feature)
  plt.show()

KeyboardInterrupt: 